In this seminar, you've explored a basic implementation of the Deep Structured Semantic Model (DSSM).

Your task is to **improve this model** in one or more of the following directions:

### ✅ Model Improvements
- Replace MLP towers with Transformer or RNN encoders or etc. (5 баллов) **(done)**
- Use different triplet loss. (3 балла) **(done)**
- Add dropout, batch normalization, or layer norm. (3 балла) **(done)**
- Integrate embeddings instead of one-hot vectors. (5 баллов) **(done)**
- Visualize similarity distribution for positive vs. negative pairs. (5 баллов)

### ✅ Evaluation & Analysis
- Visualize embeddings using t-SNE or UMAP. (3 баллов)
- Develop and improve beyond accuracy metrics. (5 баллов)

### 📄 Deliverables
- Explain what you changed and why in the final markdown cell. (3 балла) **(done)**
- Keep code modular, clean, and well-documented. (3 балла) **i hope (done)**

### 📝 Production
- create service based on DSSM vectors with ANN. (8 баллов) **(done)**

### 📝 Leaderboard
- Improve score from UserKNN via DSSM (8 баллов) **(done)**


Максимум баллов, которые можно получить - 25.

In [ ]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import requests
import torch
from tqdm import tqdm
import zipfile
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from lightning import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
import lightning as L
import pickle
import nmslib

## Предобработка данных и создание датасета

In [2]:
def download_and_extract():
    url = "https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip"
    filename = "kion_train.zip"

    response = requests.get(url, stream=True)
    with open(filename, "wb") as f:
        total = int(response.headers.get("content-length", 0))
        progress = tqdm(
            response.iter_content(1024 * 1024), f"Downloading {filename}", total=total // (1024 * 1024), unit="MB"
        )
        for chunk in progress:
            f.write(chunk)

    with zipfile.ZipFile(filename, "r") as zip_ref:
        zip_ref.extractall("data")
    os.remove(filename)


if not os.path.exists("data/data_original"):
    download_and_extract()

In [3]:
interactions = pd.read_csv("data/data_original/interactions.csv")
users = pd.read_csv("data/data_original/users.csv")
items = pd.read_csv("data/data_original/items.csv")

In [4]:
# user_cat_feats = ["age", "income", "sex", "kids_flg"]
# users_df = users.user_id
# for feat in user_cat_feats:
#     ohe_feat_df = pd.get_dummies(users[feat], prefix=feat)
#     users_df = pd.concat([users_df, ohe_feat_df], axis=1)
# users_df.head()

# убрала OHE, чтобы потом создать эмбеддинги фичей в модели
user_cat_feats = ["age", "income", "sex", "kids_flg"]
user_encoders = {}
users_df = users[["user_id"]].copy()
for feat in user_cat_feats:
    user_le = LabelEncoder()
    users_df[feat] = user_le.fit_transform(users[feat].astype(str))
    user_encoders[feat] = {class_label: index for index, class_label in enumerate(user_le.classes_)}
USER_CATEGORIES_N_UNIQUE = [users_df[col].nunique() for col in user_cat_feats]
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,1,4,2,1
1,962099,0,2,2,0
2,1047345,3,3,1,0
3,721985,3,2,1,0
4,704055,2,4,1,0


In [5]:
# item_cat_feats = ["content_type", "release_year", "for_kids", "age_rating", "studios", "countries", "directors"]
# items_df = items.item_id
# for feat in item_cat_feats:
#     ohe_feat_df = pd.get_dummies(items[feat], prefix=feat)
#     items_df = pd.concat([items_df, ohe_feat_df], axis=1)
# items_df.head()

item_cat_feats = ["content_type", "release_year", "for_kids", "age_rating", "studios", "countries", "directors"]
item_encoders = {}
items_df = items[["item_id"]].copy()
for feat in item_cat_feats:
    item_le = LabelEncoder()
    items_df[feat] = item_le.fit_transform(items[feat].astype(str))
    item_encoders[feat] = {class_label: index for index, class_label in enumerate(item_le.classes_)}

ITEM_CATEGORIES_N_UNIQUE = [items_df[col].nunique() for col in item_cat_feats]
items_df.head()

,item_id,content_type,release_year,for_kids,age_rating,studios,countries,directors
0,10711,0,85,2,2,33,258,5671
1,2508,0,97,2,2,33,421,6546
2,10716,0,94,2,2,33,298,95
3,7868,0,98,2,2,33,57,7735
4,16268,0,61,2,1,34,419,1544


In [6]:
print(f"N users before: {interactions.user_id.nunique()}")
print(f"N items before: {interactions.item_id.nunique()}\n")

interactions_df = interactions[interactions.watched_pct > 10]
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


In [7]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_df = items_df[items_df.item_id.isin(common_items)]
users_df = users_df[users_df.user_id.isin(common_users)]

65974
6901


In [8]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_df = items_df[items_df.item_id.isin(common_items)]
users_df = users_df[users_df.user_id.isin(common_users)]

65974
6897


In [9]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72.0,10616,3944
1,699317,1659,2021-05-29,8317,100.0,42131,675
6,1016458,354,2021-08-14,1672,25.0,61024,139
7,884009,693,2021-08-04,703,14.0,53150,279
14,5324,8437,2021-04-18,6598,92.0,310,3485


In [10]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), interactions_df.iid.nunique()))

for user_id, item_id in zip(interactions_df.uid, interactions_df.iid):
    interactions_vec[user_id, item_id] += 1


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

print(interactions_df.item_id.nunique())
print(items_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_df.user_id.nunique())

print(set(items_df.item_id.unique()) - set(interactions_df.item_id.unique()))

6897
6897
65974
65974
set()


In [11]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

items_df["iid"] = items_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_df = items_df.set_index("iid")

users_df["uid"] = users_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_df = users_df.set_index("uid")

In [12]:
ITEM_MODEL_SHAPE = items_df.drop(["item_id"], axis=1).shape[1]
USER_META_MODEL_SHAPE = users_df.drop(["user_id"], axis=1).shape[1]

USER_INTERACTION_MODEL_SHAPE = interactions_vec.shape[1]

print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

ITEM_MODEL_SHAPE: 7
USER_META_MODEL_SHAPE: 4
USER_INTERACTION_MODEL_SHAPE: 6897


In [ ]:
# немного ускорила код, лучше без pandas и iloc в __getitem__
class RecSysDataset(Dataset):
    def __init__(self, items, users, interactions):
        self.items = items.values
        self.users = users.values
        self.interactions = interactions
        self.num_interactions = self.interactions.shape[1]

    def __len__(self):
        return self.interactions.shape[0]

    def __getitem__(self, idx):
        p = self.interactions[idx]
        pos_i = np.random.choice(self.num_interactions, p=p)
        neg_i = np.random.randint(self.num_interactions)

        idx_meta = self.users[idx]
        idx_interaction = self.interactions[idx]
        pos = self.items[pos_i]
        neg = self.items[neg_i]

        return (
            torch.tensor(idx_meta, dtype=torch.long),
            torch.tensor(idx_interaction, dtype=torch.float32),
            torch.tensor(pos, dtype=torch.long),
            torch.tensor(neg, dtype=torch.long),
        )

In [14]:
dataset = RecSysDataset(
    items=items_df.drop(["item_id"], axis=1),
    users=users_df.drop(["user_id"], axis=1),
    interactions=interactions_vec,
)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

## Модификация модели


In [ ]:
# Transformer для признаков
# добавила также dropout, layer norm
class TabularFeatureTransformer(nn.Module):
    def __init__(self, categories_n_uniques, embed_dim=128, n_heads=2, ff_dim=256, num_layers=2, dropout=0.1):
        """
        categories_n_uniques: List[int] — количество уникальных значений для каждого категориального признака
        """
        super().__init__()

        self.num_features = len(categories_n_uniques)

        # создаём embedding слой для каждого признака
        self.embeddings = nn.ModuleList(
            [
                nn.Embedding(num_embeddings=n + 1, embedding_dim=embed_dim)  # +1 на случай OOV
                for n in categories_n_uniques
            ]
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=n_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True,  # теперь можно не транспонировать
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.layernorm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        # x: (batch_size, num_features)
        embeddings = [emb(x[:, i]) for i, emb in enumerate(self.embeddings)]
        x = torch.stack(embeddings, dim=1)  # (batch_size, num_features, embed_dim)
        x = self.encoder(x)  # (batch_size, num_features, embed_dim)
        x = self.layernorm(x.mean(dim=1))  # агрегируем по признакам
        return x  # (batch_size, embed_dim)

In [ ]:
# попытка сделать трансформер для последовательности истории, пока неудачная
# class TabularHistTransformer(nn.Module):
#     def __init__(self, input_dim, embed_dim=128, n_heads=2, ff_dim=256, num_layers=2, dropout=0.1):
#         super().__init__()
#         self.input_proj = nn.Linear(1, embed_dim)

#         encoder_layer = nn.TransformerEncoderLayer(
#             d_model=embed_dim,
#             nhead=n_heads,
#             dim_feedforward=ff_dim,
#             dropout=dropout,
#             batch_first=True,
#         )
#         self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
#         self.layernorm = nn.LayerNorm(embed_dim)

#     def forward(self, x):
#         # x: (batch_size, feature_dim)
#         x = x.unsqueeze(-1)  # (batch_size, feature_dim, 1)
#         x = self.input_proj(x)  # (batch_size, feature_dim, embed_dim)
#         x = self.encoder(x)  # (batch_size, feature_dim, embed_dim)
#         x = self.layernorm(x.mean(dim=1))  # агрегируем по фичам
#         return x  # (batch_size, embed_dim)

In [17]:
class DSSMTransformer(nn.Module):
    def __init__(self, user_meta_dim, user_hist_dim, item_feat_dim, embed_dim=128):
        super().__init__()
        # отдельные энкодеры для каждой части
        self.user_meta_encoder = TabularFeatureTransformer(user_meta_dim, embed_dim)
        # self.user_hist_encoder = TabularHistTransformer(user_hist_dim, embed_dim)
        # пока не получилось отладить трансформерную версию,
        # поэтому для кодирования истории оставлю линейный слой
        self.user_hist_encoder = nn.Linear(user_hist_dim, embed_dim)
        self.item_encoder = TabularFeatureTransformer(item_feat_dim, embed_dim)

        # финальное объединение user части
        self.user_fc = nn.Linear(embed_dim * 2, embed_dim)

    def encode_user(self, meta, hist):
        meta_emb = self.user_meta_encoder(meta)
        hist_emb = torch.relu(self.user_hist_encoder(hist))
        user_emb = torch.cat([meta_emb, hist_emb], dim=1)
        return F.normalize(self.user_fc(user_emb))

    def encode_item(self, item_feat):
        return F.normalize(self.item_encoder(item_feat))

    def forward(self, user_meta, user_hist, item_feat):
        user_emb = self.encode_user(user_meta, user_hist)
        item_emb = self.encode_item(item_feat)
        # sim = F.cosine_similarity(user_emb, item_emb)
        # решила использовать dot product, так как он чаще используется с BPR,
        # чем cosine similarity
        sim = (user_emb * item_emb).sum(dim=1)
        return sim

In [18]:
# изменила лосс на BPR
class DSSMRecSysModel(L.LightningModule):
    def __init__(self, user_meta_dim, user_hist_dim, item_feat_dim, embed_dim=128, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()

        self.dssm = DSSMTransformer(user_meta_dim, user_hist_dim, item_feat_dim, embed_dim)
        self.lr = lr

    def forward(self, user_meta, user_hist, item):
        return self.dssm(user_meta, user_hist, item)

    def training_step(self, batch, batch_idx):
        user_meta, user_hist, pos_item, neg_item = batch
        pos_sim = self(user_meta, user_hist, pos_item)
        neg_sim = self(user_meta, user_hist, neg_item)

        # BPR loss
        diff = pos_sim - neg_sim
        loss = -F.logsigmoid(diff).mean()
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        if batch_idx % 500 == 0:
            print(f"{loss}")
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def get_user_embeddings(self, meta, hist):
        return self.dssm.encode_user(meta, hist)

    def get_item_embeddings(self, item_feat):
        return self.dssm.encode_item(item_feat)

In [22]:
model = DSSMRecSysModel(
    user_meta_dim=USER_CATEGORIES_N_UNIQUE,
    user_hist_dim=USER_INTERACTION_MODEL_SHAPE,
    item_feat_dim=ITEM_CATEGORIES_N_UNIQUE,
    embed_dim=64,
    lr=1e-3,
)

In [ ]:
loss_checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/",
    filename="best-checkpoint",
    monitor="train_loss",
    mode="min",
    save_top_k=1,
)

trainer = Trainer(
    callbacks=[loss_checkpoint_callback],
    max_epochs=20,
    devices=1,
    accelerator="cuda",
    log_every_n_steps=100,
)

# по хорошему, нужно отделить датасет для валидации конечно...
# но пока успеваю только так
trainer.fit(model, dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type            | Params | Mode 
-------------------------------------------------
0 | dssm | DSSMTransformer | 1.2 M  | train
-------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.865     Total estimated model params size (MB)
64        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

0.696000337600708
0.5497417449951172
0.4905659258365631
0.4710859954357147
0.4150999188423157
0.4272647798061371
0.47055330872535706
0.3870401978492737
0.49417683482170105
0.41589730978012085
0.47959473729133606
0.36399340629577637
0.3950423002243042
0.41561681032180786
0.3479180932044983
0.35648202896118164
0.4048798680305481
0.3624531030654907
0.43366873264312744
0.3782428801059723
0.37151482701301575
0.34560877084732056
0.3598695397377014
0.42778342962265015
0.3292683959007263
0.33185288310050964
0.4018973112106323
0.40156006813049316
0.37271010875701904
0.4062381386756897
0.35872840881347656
0.4079963266849518
0.3697158992290497
0.36353522539138794
0.3614277243614197
0.4134209156036377
0.40662258863449097
0.3986271321773529
0.38601821660995483
0.37435853481292725


`Trainer.fit` stopped: `max_epochs=20` reached.


## Сохранение артефактов ANN для сервиса

In [ ]:
model.eval()

user_meta_tensor = torch.tensor(users_df.drop("user_id", axis=1).values, dtype=torch.long)
user_hist_tensor = torch.tensor(interactions_vec, dtype=torch.float32)

with torch.no_grad():
    user_vectors = model.get_user_embeddings(user_meta_tensor, user_hist_tensor).cpu().numpy()

item_feat_tensor = torch.tensor(items_df.drop("item_id", axis=1).values, dtype=torch.long)

with torch.no_grad():
    item_vectors = model.get_item_embeddings(item_feat_tensor).cpu().numpy()

user_ids = users_df["user_id"].values
item_ids = items_df["item_id"].values

user_id_map = {i: uid for i, uid in enumerate(user_ids)}
item_id_map = {i: iid for i, iid in enumerate(item_ids)}
reverse_user_id_map = {uid: i for i, uid in user_id_map.items()}

save_dir = "to_recsys"
np.save(f"{save_dir}/user_vectors.npy", user_vectors)
np.save(f"{save_dir}/item_vectors.npy", item_vectors)

with open(f"{save_dir}/user_id_map.pkl", "wb") as f:
    pickle.dump(user_id_map, f)
with open(f"{save_dir}/item_id_map.pkl", "wb") as f:
    pickle.dump(item_id_map, f)
with open(f"{save_dir}/reverse_user_id_map.pkl", "wb") as f:
    pickle.dump(reverse_user_id_map, f)

index = nmslib.init(method="hnsw", space="negdotprod")
index.addDataPointBatch(item_vectors)
index.createIndex({"post": 2}, print_progress=True)

index.saveIndex(f"{save_dir}/ann_index.nmslib")

In [ ]:
# просто примерно проверила, как будет в сервисе загружаться
load_dir = "to_recsys"

user_vectors = np.load(f"{load_dir}/user_vectors.npy")
item_vectors = np.load(f"{load_dir}/item_vectors.npy")

with open(f"{load_dir}/user_id_map.pkl", "rb") as f:
    user_id_map = pickle.load(f)
with open(f"{load_dir}/item_id_map.pkl", "rb") as f:
    item_id_map = pickle.load(f)
with open(f"{load_dir}/reverse_user_id_map.pkl", "rb") as f:
    reverse_user_id_map = pickle.load(f)

index = nmslib.init(method="hnsw", space="cosinesimil")
index.loadIndex(f"{load_dir}/ann_index.nmslib")


def recommend(user_id, top_n=10):
    if user_id not in reverse_user_id_map:
        raise ValueError(f"Unknown user_id: {user_id}")
    user_idx = reverse_user_id_map[user_id]
    user_vector = user_vectors[user_idx]
    ids, distances = index.knnQuery(user_vector, k=top_n)
    return [item_id_map[i] for i in ids]

In [64]:
recommend(2)

[14391, 15479, 8251, 9351, 10447, 7036, 5978, 6028, 10612, 12247]

## Что было сделано
1. Код предобработки данных по большей части взяла из семинара, изменила только то, как кодируются категориальные признаки. Для получения эмбеддингов категорий нужно Label кодирование, а не One Hot. Это позволило мне далее использовать трансформеры 

2. Немного оптимизировала класс датасета, так как у меня он очень медленно работал. Избавилась от pandas, что все ускорило

3. Главные изменения были в модели. Мне очень хотелось использовать трансформерные модели на табличках, поэтому написала свой базовый класс, вдохновленный TabTransformer. Так как у нас все признаки категориальные, то каждая уникальная категория получает эмбеддинг, тогда юзер или айтем это последовательность таких эмбеддингов категорий. Специально не стала добавлять позиционные эмбеддинги, так как не важен порядок категорий. И сверху всей этой красоты просто self attention

4. Много провозилась с тем, чтобы использовать трансформер и для interactions, но у меня либо ничего не училось, либо училось плохо, поэтому просто оставила линейный слой, уже хорошо себя показавший

5. Заменила лосс на BPR

6. Добавила layer normalisation, dropout для стабилизации обучения

7. На лидерборде появилась с моделью 'dssm' и MAP@10 = 0.0880846 (у моей UserNNN было 0.07441776)

8. Гиперпараметры, честно говоря, подбирала эмпирически и на глаз из-за нехватки времени. При увеличении той же размерности эмбеддингов модель начинала учиться дольше и в какой-то момент замирала на плато, лосс почти не падал. В теории, это можно решить каким-нибудь динамическим подбором lr, но это пунктик на будущее 